In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Data Science Level 2 Practical Problem 2305-1

In [2]:
df = pd.read_csv('lv2-2305-1.csv')

#### Q1

In [3]:
df4 = df.copy()

In [4]:
from sklearn.preprocessing import StandardScaler
# Step 1-1 
sc = StandardScaler()
df4.iloc[:,1:5] = sc.fit_transform(df4.iloc[:,1:5])

#Step 1-2
df4['INVOLVEMENT'] = df4['NUM_LOGIN'] + df4['NUM_CONTENTS'] + df4['NUM_ACTION'] + df4['TIME_LOGIN']

#Step 1-3 
from scipy.stats import pearsonr
coef,p=pearsonr(df4['INVOLVEMENT'],df4['SCORE'])

In [5]:
print('Q1 Result:', coef)
print('Q1 Answer:', round(coef, 3))

Q1 Result: 0.44598688704115624
Q1 Answer: 0.446


#### Q2

In [6]:
df5 = df.copy()

In [7]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Step 2-1
q1 = df5['SCORE'].quantile(0.25)
q3 = df5['SCORE'].quantile(0.75)
iqr = q3-q1
df5_1 = df5[(df5['SCORE'] >= q1 -1.5*iqr) & (df5['SCORE'] <= q3 + 1.5*iqr)]

# Step 2-2
train = df5_1[df5_1['EMP_ID'].str[4].astype('int') % 4 != 0]
test = df5_1[df5_1['EMP_ID'].str[4].astype('int') % 4 == 0]

# Step 2-3
y = train[['SCORE']]
max_num = -999
f_list= ['NUM_LOGIN', 'NUM_CONTENTS', 'NUM_ACTION', 'TIME_LOGIN', 'TIME_TEST', 'NUM_REVISION']
for f in f_list:
    x = train[[f]]
    lr = LinearRegression()
    lr.fit(x,y)
    adj_r = 1 - (1-lr.score(x,y))*(len(y)-1)/(len(y)-x.shape[1]-1)
    if adj_r > max_num:
        max_num = adj_r
        max_var = f
print(max_num, max_var)    

0.26585408702965907 TIME_LOGIN


In [8]:
y = train[['SCORE']]
max_num = -999
f_list= ['NUM_LOGIN', 'NUM_CONTENTS', 'NUM_ACTION', 'TIME_TEST', 'NUM_REVISION'] # Remove TIME_LOGIN
for f in f_list:
    x = train[['TIME_LOGIN',f]]
    lr = LinearRegression()
    lr.fit(x,y)
    adj_r = 1 - (1-lr.score(x, y))*(len(y)-1)/(len(y)-x.shape[1]-1)
    if adj_r > max_num:
        max_num = adj_r
        max_var = f
print(max_num, max_var)    

0.29674428713721424 NUM_CONTENTS


In [9]:
y = train[['SCORE']]
max_num = -999
f_list= ['NUM_LOGIN', 'NUM_ACTION', 'TIME_TEST', 'NUM_REVISION']
for f in f_list:
    x = train[['TIME_LOGIN','NUM_CONTENTS',f]]
    lr = LinearRegression()
    lr.fit(x,y)
    adj_r = 1 - (1-lr.score(x, y))*(len(y)-1)/(len(y)-x.shape[1]-1)
    if adj_r > max_num:
        max_num = adj_r
        max_var = f
print(max_num, max_var)    

0.3265951102870196 NUM_REVISION


In [10]:
# Step 2-4
from sklearn.metrics import mean_squared_error
train_x = train[['TIME_LOGIN', 'NUM_CONTENTS', 'NUM_REVISION']]
train_y = train[['SCORE']]
test_x = test[['TIME_LOGIN', 'NUM_CONTENTS', 'NUM_REVISION']]
test_y = test[['SCORE']]
# Sklearn 
lr = LinearRegression()
lr.fit(train_x,train_y)

test_y['prd'] = lr.predict(test_x)
res = mean_squared_error(test_y['prd'], test_y['SCORE']) **0.5

In [11]:
print('Q2 Result:', res)
print('Q2 Answer:', round(res, 2))

Q2 Result: 1.3919187141780969
Q2 Answer: 1.39


#### Q3

In [12]:
df6 = df.copy()

In [13]:
# Step 3-1
df6_a = df6[df6['SCORE'] <= 5]
df6_b = df6[(df6['SCORE'] <=7) & (df6['SCORE']>5)]
df6_c = df6[(df6['SCORE'] <=10) & (df6['SCORE']>7)]

In [14]:
from sklearn.ensemble import RandomForestRegressor

# Step 3-2
rf_a = RandomForestRegressor(random_state=1234, n_estimators = 10, min_samples_leaf = 10)
rf_b = RandomForestRegressor(random_state=1234, n_estimators = 10, min_samples_leaf = 10)
rf_c = RandomForestRegressor(random_state=1234, n_estimators = 10, min_samples_leaf= 10)

train_a_y = df6_a['SCORE']
train_a_x = df6_a.drop(columns=['EMP_ID','SCORE'])

train_b_y = df6_b['SCORE']
train_b_x = df6_b.drop(columns=['EMP_ID','SCORE'])

train_c_y = df6_c['SCORE']
train_c_x = df6_c.drop(columns=['EMP_ID','SCORE'])

rf_a.fit(train_a_x, train_a_y)
a_fi = rf_a.feature_importances_

rf_b.fit(train_b_x, train_b_y)
b_fi = rf_b.feature_importances_

rf_c.fit(train_c_x, train_c_y)
c_fi = rf_c.feature_importances_

In [15]:
# Step 3-3 Compare Feature Importance 

In [16]:
a_fi

array([0.1       , 0.28329769, 0.6       , 0.        , 0.        ,
       0.01670231])

In [17]:
b_fi

array([0.02380331, 0.00271996, 0.03890917, 0.74317753, 0.16952741,
       0.02186262])

In [18]:
c_fi

array([0.        , 0.0074127 , 0.17934068, 0.10480842, 0.6058609 ,
       0.1025773 ])

In [19]:
res = a_fi.max() + b_fi.max() + c_fi.max()

In [20]:
print('Q3 Result:', res)
print('Q3 Answer:', round(res, 2))

Q3 Result: 1.9490384300673917
Q3 Answer: 1.95
